In [1]:
import pandas as pd
import os
import re
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [53]:
import keras_vggface
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input

print(keras_vggface.__version__)

import keras.utils as image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error

#from keras.metrics import PearsonCorrelationCoefficient
from tensorflow.keras.metrics import MeanAbsoluteError

from scipy.stats import pearsonr
from tensorflow.keras import backend as K


0.6


### <strong>Define model</strong>

In [25]:
base_model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False
    
#base_model.summary()

In [4]:
import tensorflow_probability as tfp
def pearson_corr(y_true, y_pred):
   
    corr = tfp.stats.correlation(y_true,y_pred) 
    return corr

tf.keras.utils.get_custom_objects()['pearson_corr'] = pearson_corr

In [26]:
base_model.summary()

Model: "vggface_resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1/7x7_s2 (Conv2D)          (None, 112, 112, 64  9408        ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 conv1/7x7_s2/bn (BatchNormaliz  (None, 112, 112, 64  256        ['conv1/7x7_s2[0][0]']           
 ation)                         )                                                  

In [68]:
# Get the output of the last layer of the ResNet50 model
x = base_model.output

# Flatten the output of the last layer to create a 1D vector
x = Flatten()(x)

# Add a dense layer with 128 neurons and 'relu' activation function
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

x = Dense(64, activation='relu')(x)
x = BatchNormalization()(x)
# x = Dropout(0.2)(x)

# Add the final dense layer with one neuron and a linear activation function
out = Dense(1, activation='linear')(x)

# Create a new model that includes the ResNet50 model and the new layers
model = Model(base_model.input, out)

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=[MeanAbsoluteError()])


#model.summary()

### <strong>Load images</strong>

In [6]:
# df_train = pd.read_json('df_train.json')
# df_test = pd.read_json('df_test.json')

In [7]:
with open('df_train.json', 'r') as f:
    json_data = f.read()

df_train = pd.read_json(json_data)

with open('df_test.json', 'r') as f:
    json_data = f.read()

df_test = pd.read_json(json_data)

In [8]:
X_train_images =  np.zeros((len(df_train), 224, 224, 3))
for i, filename in enumerate(df_train['name']):
    img = load_img('/home/jupyter/ML Final/Images/'+filename, target_size = (224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    img = preprocess_input(img, version=2)
    #image = Image.open(path + filename)
    #image = image.resize((224, 224))
    X_train_images[i] = np.array(img)
    
X_test_images =  np.zeros((len(df_test), 224, 224, 3))
for i, filename in enumerate(df_test['name']):
    img = load_img('/home/jupyter/ML Final/Images/'+filename, target_size = (224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    img = preprocess_input(img, version=2)
    #image = Image.open(path + filename)
    #image = image.resize((224, 224))
    X_test_images[i] = np.array(img)

In [9]:
y_train_bmi = np.array(df_train['bmi'])
y_test_bmi = np.array(df_test['bmi'])

In [10]:
print(X_train_images.shape)
print(X_test_images.shape)
print(y_train_bmi.shape)
print(y_test_bmi.shape)

(3210, 224, 224, 3)
(752, 224, 224, 3)
(3210,)
(752,)


In [69]:
history = model.fit(X_train_images, y_train_bmi, batch_size=32, epochs=10, validation_data=(X_test_images, y_test_bmi))

Epoch 1/10
101/101 [==============================] - 83s 782ms/step - loss: 1049.6697 - mean_absolute_error: 31.7879 - val_loss: 1192.1902 - val_mean_absolute_error: 33.7528
Epoch 2/10
101/101 [==============================] - 78s 771ms/step - loss: 870.5201 - mean_absolute_error: 28.9397 - val_loss: 788.1429 - val_mean_absolute_error: 27.0945
Epoch 3/10
101/101 [==============================] - 78s 771ms/step - loss: 575.0264 - mean_absolute_error: 23.3410 - val_loss: 525.2796 - val_mean_absolute_error: 21.7561
Epoch 4/10
101/101 [==============================] - 78s 773ms/step - loss: 283.9614 - mean_absolute_error: 16.0212 - val_loss: 266.9643 - val_mean_absolute_error: 14.5906
Epoch 5/10
101/101 [==============================] - 77s 766ms/step - loss: 104.5166 - mean_absolute_error: 9.0661 - val_loss: 114.2967 - val_mean_absolute_error: 8.4383
Epoch 6/10
101/101 [==============================] - 78s 776ms/step - loss: 36.9543 - mean_absolute_error: 4.8580 - val_loss: 73.1855 

In [59]:
y_pred = model.predict(X_test_images)

corr, _ = pearsonr(y_test_bmi, y_pred)
print("Test correlation coefficient:", corr) #batch normlization 10 epochs 1 drop out 2 norml mae 0.001

24/24 [==============================] - 15s 611ms/step
Test correlation coefficient: [0.61006355660211]


In [64]:
y_pred = model.predict(X_test_images)

corr, _ = pearsonr(y_test_bmi, y_pred)
print("Test correlation coefficient:", corr) #batch normlization 10 epochs 1 drop out 2 norml corr 0.001

24/24 [==============================] - 16s 625ms/step
Test correlation coefficient: [0.6084098279695221]


In [70]:
y_pred = model.predict(X_test_images)

corr, _ = pearsonr(y_test_bmi, y_pred)
print("Test correlation coefficient:", corr) #batch normlization 10 epochs 1 drop out 2 norml mae 0.001

24/24 [==============================] - 15s 608ms/step
Test correlation coefficient: [0.6155155985360238]


In [72]:
ivy = load_img('/home/jupyter/ML Final/Images/img_0.bmp', target_size = (224, 224))
ivy = image.img_to_array(ivy)
ivy = np.expand_dims(ivy, axis = 0)
ivy = preprocess_input(ivy, version=2)

ivy = np.array(ivy)
model.predict(ivy)

1/1 [==============================] - 0s 64ms/step


array([[34.42149]], dtype=float32)

In [73]:
# Calculate MAE
mae = mean_absolute_error(y_test_bmi, y_pred)
mae

5.459330011271344

In [74]:
model.save('customized_model.h5')